In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2

from tqdm import tqdm
from glob import glob
import os
import json 
import timm

import torch
from torch import nn
from torchvision import models
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary
# from torchsampler import ImbalancedDatasetSampler

from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split, StratifiedKFold
import albumentations as A

# Train dataset 준비

In [2]:
# 변수 설명 csv 파일 참조
crop = {'1':'딸기','2':'토마토','3':'파프리카','4':'오이','5':'고추','6':'시설포도'}
disease = {'1':{'a1':'딸기잿빛곰팡이병','a2':'딸기흰가루병','b1':'냉해피해','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '2':{'a5':'토마토흰가루병','a6':'토마토잿빛곰팡이병','b2':'열과','b3':'칼슘결핍','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '3':{'a9':'파프리카흰가루병','a10':'파프리카잘록병','b3':'칼슘결핍','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '4':{'a3':'오이노균병','a4':'오이흰가루병','b1':'냉해피해','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '5':{'a7':'고추탄저병','a8':'고추흰가루병','b3':'칼슘결핍','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '6':{'a11':'시설포도탄저병','a12':'시설포도노균병','b4':'일소피해','b5':'축과병'}}
risk = {'1':'초기','2':'중기','3':'말기'}

In [3]:
label_description = {}
for key, value in disease.items():
    label_description[f'{key}_00_0'] = f'{crop[key]}_정상'
    for disease_code in value:
        for risk_code in risk:
            label = f'{key}_{disease_code}_{risk_code}'
            label_description[label] = f'{crop[key]}_{disease[key][disease_code]}_{risk[risk_code]}'
list(label_description.items())[:10]

[('1_00_0', '딸기_정상'),
 ('1_a1_1', '딸기_딸기잿빛곰팡이병_초기'),
 ('1_a1_2', '딸기_딸기잿빛곰팡이병_중기'),
 ('1_a1_3', '딸기_딸기잿빛곰팡이병_말기'),
 ('1_a2_1', '딸기_딸기흰가루병_초기'),
 ('1_a2_2', '딸기_딸기흰가루병_중기'),
 ('1_a2_3', '딸기_딸기흰가루병_말기'),
 ('1_b1_1', '딸기_냉해피해_초기'),
 ('1_b1_2', '딸기_냉해피해_중기'),
 ('1_b1_3', '딸기_냉해피해_말기')]

In [4]:
csv_feature_dict = {
    '내부 온도 1 평균': [3.4, 47.3],
    '내부 온도 1 최고': [3.4, 47.6],
    '내부 온도 1 최저': [3.3, 47.0],
    '내부 습도 1 평균': [23.7, 100.0],
    '내부 습도 1 최고': [25.9, 100.0],
    '내부 습도 1 최저': [0.0, 100.0],
    '내부 이슬점 평균': [0.1, 34.5],
    '내부 이슬점 최고': [0.2, 34.7],
    '내부 이슬점 최저': [0.0, 34.4]
}

In [5]:
# ============= add
labels = pd.read_csv('./data/train.csv')

train_label_encoder = {}
label_cnt = 0
previous_label = '0_00_0'
for i, label in enumerate(tqdm(sorted(labels['label']))) :
    crop_val = label.split('_')[0] # crop
    disease_val = label.split('_')[1] # disease
    risk_val = label.split('_')[2] # risk
    
    tmp_label = f'{crop_val}_{disease_val}_{risk_val}'
    if previous_label != tmp_label :
        train_label_encoder[tmp_label] = label_cnt
        previous_label = tmp_label
        label_cnt += 1
        
train_label_decoder = {val : key for key, val in train_label_encoder.items()}
display(train_label_decoder)
display(train_label_encoder)

100%|██████████████████████████████████████████████████████████████████████████| 5767/5767 [00:00<00:00, 963764.09it/s]


{0: '1_00_0',
 1: '2_00_0',
 2: '2_a5_2',
 3: '3_00_0',
 4: '3_a9_1',
 5: '3_a9_2',
 6: '3_a9_3',
 7: '3_b3_1',
 8: '3_b6_1',
 9: '3_b7_1',
 10: '3_b8_1',
 11: '4_00_0',
 12: '5_00_0',
 13: '5_a7_2',
 14: '5_b6_1',
 15: '5_b7_1',
 16: '5_b8_1',
 17: '6_00_0',
 18: '6_a11_1',
 19: '6_a11_2',
 20: '6_a12_1',
 21: '6_a12_2',
 22: '6_b4_1',
 23: '6_b4_3',
 24: '6_b5_1'}

{'1_00_0': 0,
 '2_00_0': 1,
 '2_a5_2': 2,
 '3_00_0': 3,
 '3_a9_1': 4,
 '3_a9_2': 5,
 '3_a9_3': 6,
 '3_b3_1': 7,
 '3_b6_1': 8,
 '3_b7_1': 9,
 '3_b8_1': 10,
 '4_00_0': 11,
 '5_00_0': 12,
 '5_a7_2': 13,
 '5_b6_1': 14,
 '5_b7_1': 15,
 '5_b8_1': 16,
 '6_00_0': 17,
 '6_a11_1': 18,
 '6_a11_2': 19,
 '6_a12_1': 20,
 '6_a12_2': 21,
 '6_b4_1': 22,
 '6_b4_3': 23,
 '6_b5_1': 24}

# Custom dataset 

In [6]:
class CustomDataset(Dataset):
    def __init__(
        self, 
        files, 
        csv_feature_dict, 
        label_encoder,
        labels=None,
        transforms=None,
        mode='train',
    ):
        self.mode = mode
        self.files = files
        
        self.csv_feature_dict = csv_feature_dict
        
        if files is not None:
            self.csv_feature_check = [0]*len(self.files)
            self.csv_features = [None]*len(self.files)
            
        self.max_len = MAX_LEN
        self.num_features = NUM_FEATURES
        
        self.label_encoder = label_encoder
        
        self.transforms = transforms

    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, i):
        file = self.files[i]
        file_name = file.split(os.sep)[-1]
        
        # csv
        csv_path = f'{file}/{file_name}.csv'
        df = pd.read_csv(csv_path)[self.csv_feature_dict.keys()]
        df = df.replace('-', 0)

        # MinMax scaling
        for col in df.columns:
            df[col] = df[col].astype(float) - self.csv_feature_dict[col][0]
            df[col] = df[col] / (self.csv_feature_dict[col][1]-self.csv_feature_dict[col][0])

        df_dict = {}
        df_dict['내부 온도 1 최저'] = np.max(df[['내부 온도 1 최저']].to_numpy(), -1)
        df_dict['내부 온도 1 평균'] = np.max(df[['내부 온도 1 평균']].to_numpy(), -1)
        df_dict['내부 온도 1 최고'] = np.max(df[['내부 온도 1 최고']].to_numpy(), -1)
        df_dict['내부 습도 1 최저'] = np.max(df[['내부 습도 1 최저']].to_numpy(), -1)
        df_dict['내부 습도 1 평균'] = np.max(df[['내부 습도 1 평균']].to_numpy(), -1)
        df_dict['내부 습도 1 최고'] = np.max(df[['내부 습도 1 최고']].to_numpy(), -1)
        df_dict['내부 이슬점 최저'] = np.max(df[['내부 이슬점 최저']].to_numpy(), -1)
        df_dict['내부 이슬점 평균'] = np.max(df[['내부 이슬점 평균']].to_numpy(), -1)
        df_dict['내부 이슬점 최고'] = np.max(df[['내부 이슬점 최고']].to_numpy(), -1)
        seq_len = len(df_dict['내부 온도 1 최저'])
        df = pd.DataFrame(df_dict)

        after_df = pd.DataFrame(df_dict).to_numpy()
        x        = np.zeros([self.max_len, self.num_features])

        h, w      = after_df.shape
        x[0:h, :] = after_df
        
        # image
        image_path = f'{file}/{file_name}.jpg'
        img = cv2.imread(image_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        ### Image Padding 생략
#         h,w,c = img.shape
#         if w < h: # width padding
#             pad_w = (h - w) // 2
#             pad_img = np.zeros([h,h,3]).astype(np.uint8)
#             pad_img[:,pad_w:pad_w+w] = img
#             img = pad_img
#         elif w > h: # height padding
#             pad_h = (w - h) // 2
#             pad_img = np.zeros([w,w,3]).astype(np.uint8)
#             pad_img[pad_h:pad_h+h,:] = img
#             img = pad_img
        
        if self.transforms is not None:
            img = self.transforms(image=img)['image']
        img = img.transpose(2,0,1)
        
        if self.mode == 'train':
            json_path = f'{file}/{file_name}.json'
            with open(json_path, 'r') as f:
                json_file = json.load(f)
            
            crop = json_file['annotations']['crop']
            disease = json_file['annotations']['disease']
            risk = json_file['annotations']['risk']
            label = f'{crop}_{disease}_{risk}'
            
            return {
                'img': torch.tensor(img, dtype=torch.float32) / 255.0,
                'csv_feature': torch.tensor(x, dtype=torch.float32),
                'seq_len' : seq_len,
                'label': torch.tensor(self.label_encoder[label], dtype=torch.long)
            }
        else:
            return {
                'img': torch.tensor(img, dtype=torch.float32) / 255.0,
                'seq_len' : seq_len,
                'csv_feature': torch.tensor(x, dtype=torch.float32)
            }

# CNN Model

In [7]:
class EffiV2S(nn.Module):
    def __init__(self):
        super(EffiV2S, self).__init__()
        self.model = timm.create_model('efficientnetv2_rw_s', pretrained=True, num_classes=38)
#         self.model.load_state_dict(torch.load(model_path, map_location=device))
        
    def forward(self, inputs):
        output = self.model(inputs)
        return output
    
class CNN_ENCODER(nn.Module) :
    def __init__(self, model_path):
        super(CNN_ENCODER, self).__init__()
        self.model = EffiV2S()
        self.model.load_state_dict(torch.load(model_path, map_location=device))
        self.model.to(device)
        self.model.model.classifier = nn.Linear(1792, 1000, bias=True).to(device)
        
    def forward(self, inputs):
        output = self.model(inputs)
        return output

### model load test

In [8]:
# data = torch.randn(1,3,288, 288, dtype=torch.float32).to(device)
# model_path = './model/public_vill_50k_pretrain_EffiV2S.pt'
# model = CNN_ENCODER('./model/public_vill_50k_pretrain_EffiV2S.pt').to(device)
# a = model(data)
# a.shape

# RNN model

In [9]:
class SELayer(nn.Module):
    def __init__(self, channel, reduction=16):
        super(SELayer, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Sequential(
            nn.Linear(channel, channel // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channel // reduction, channel, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1)
        return x * y.expand_as(x)

class MLSTMfcn(nn.Module):
    def __init__(self, *, num_classes, max_seq_len, num_features,
                 num_lstm_out=128, num_lstm_layers=1, 
                 conv1_nf=128, conv2_nf=256, conv3_nf=128,
                 lstm_drop_p=0.8, fc_drop_p=0.3):
        
        super(MLSTMfcn, self).__init__()
        self.num_classes = num_classes
        self.max_seq_len = max_seq_len
        self.num_features = num_features

        self.num_lstm_out = num_lstm_out
        self.num_lstm_layers = num_lstm_layers

        self.conv1_nf = conv1_nf
        self.conv2_nf = conv2_nf
        self.conv3_nf = conv3_nf

        self.lstm_drop_p = lstm_drop_p
        self.fc_drop_p = fc_drop_p

        self.lstm = nn.LSTM(input_size=self.num_features, 
                            hidden_size=self.num_lstm_out,
                            num_layers=self.num_lstm_layers,
                            batch_first=True)
        
        self.conv1 = nn.Conv1d(self.num_features, self.conv1_nf, 8)
        self.conv2 = nn.Conv1d(self.conv1_nf, self.conv2_nf, 5)
        self.conv3 = nn.Conv1d(self.conv2_nf, self.conv3_nf, 3)

        self.bn1 = nn.BatchNorm1d(self.conv1_nf)
        self.bn2 = nn.BatchNorm1d(self.conv2_nf)
        self.bn3 = nn.BatchNorm1d(self.conv3_nf)

        self.se1 = SELayer(self.conv1_nf)  # ex 128
        self.se2 = SELayer(self.conv2_nf)  # ex 256

        self.relu = nn.ReLU()
        self.lstmDrop = nn.Dropout(self.lstm_drop_p)
        self.convDrop = nn.Dropout(self.fc_drop_p)

        self.fc = nn.Linear(self.conv3_nf+self.num_lstm_out, 128)

        self.out_layer = nn.Linear(1000+128, self.num_classes)
        self.dropout = nn.Dropout(0.1)
    
    def forward(self, enc_out, x, seq_lens):
        ''' input x should be in size [B,T,F], where 
            B = Batch size
            T = Time samples
            F = features
        '''
        x1 = nn.utils.rnn.pack_padded_sequence(x, seq_lens.cpu(), 
                                               batch_first=True, 
                                               enforce_sorted=False)
        x1, (ht,ct) = self.lstm(x1)
        x1, _ = nn.utils.rnn.pad_packed_sequence(x1, batch_first=True, 
                                                 padding_value=0.0)
        x1 = x1[:,-1,:]
        
        x2 = x.transpose(2,1)
        x2 = self.convDrop(self.relu(self.bn1(self.conv1(x2))))
        x2 = self.se1(x2)
        x2 = self.convDrop(self.relu(self.bn2(self.conv2(x2))))
        x2 = self.se2(x2)
        x2 = self.convDrop(self.relu(self.bn3(self.conv3(x2))))
        x2 = torch.mean(x2,2)
        
        x_all = torch.cat((x1,x2),dim=1)
        x_out = self.fc(x_all)
        concat = torch.cat([enc_out, x_out], dim=1)  # enc_out + hidden 
        output = self.dropout(concat)
        x_output = self.out_layer(output)
        x_out = F.log_softmax(x_output, dim=1)

        return x_out

# CNN + RNN 

In [10]:
class CNN2RNNModel(nn.Module):
    def __init__(
        self,
        model_path,
        n_classes
    ):
        super(CNN2RNNModel, self).__init__()
        
        self.cnn = CNN_ENCODER(model_path)
        self.rnn = MLSTMfcn(num_classes=n_classes, max_seq_len=MAX_LEN, num_features=NUM_FEATURES)


    def forward(self, img, seq, seq_len):
        cnn_output = self.cnn(img)
        output = self.rnn(cnn_output, seq, seq_len)
        
        return output

# training

In [29]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)
 
    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

In [12]:
def accuracy_function(real, pred):    
    real = real.cpu()
    pred = torch.argmax(pred, dim=1).cpu()
    score = f1_score(real, pred, average='macro')
    return score

def train_step(batch_item, training):
    img = batch_item['img'].to(device)
    csv_feature  = batch_item['csv_feature'].to(device)
    seq_len  = batch_item['seq_len'].to(device)
    label = batch_item['label'].to(device)

    lam = np.random.beta(1.0, 1.0)
    
    if training is True:
        model.train()
        optimizer.zero_grad()
        with torch.cuda.amp.autocast():
            # add - cutmix
            rand_index = torch.randperm(img.size()[0])
            target_a = label
            target_b = label[rand_index]
            bbx1, bby1, bbx2, bby2 = rand_bbox(img.size(), lam)
            img[:, :, bbx1:bbx2, bby1:bby2] = img[rand_index, :, bbx1:bbx2, bby1:bby2]
            lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (img.size()[-1] * img.size()[-2]))
            
            output = model(img, csv_feature, seq_len)
#             loss = criterion(output, label)
            loss = criterion(output, target_a) * lam + criterion(output, target_b) * (1. - lam)
            
        loss.backward()
        optimizer.step()
        score = accuracy_function(label, output)
        return loss, score
    
    else:
        model.eval()
        with torch.no_grad():
            output = model(img, csv_feature, seq_len)
            loss = criterion(output, label)
        score = accuracy_function(label, output)
        return loss, score

# Hyperparameter

In [11]:
batch_size = 16
n_classes = len(train_label_encoder)
learning_rate = 1e-3
NUM_FEATURES  = len(csv_feature_dict)
MAX_LEN  = 590
DROPOUT_RATE  = 0.1
epochs = 25
device = torch.device("cuda:0")#("cpu")
early_stopping_cnt = 4
save_path = 'public_vill_50k_pretrain_Effi_1000_CNN2RNN.pt'
model_path = './model/public_vill_50k_pretrain_EffiV2S.pt'
fold_n = 4

# model이랑 loss 선언

In [14]:
model = CNN2RNNModel(model_path, n_classes)
model = model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=5e-2)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20, eta_min=0.0001) 
criterion = nn.CrossEntropyLoss()

In [12]:
json_path = glob('./data/train/*/*.json')

label_list = []
for path in tqdm(json_path) :
    json_file = json.load(open(path, 'r'))
    
    crop = json_file['annotations']['crop']
    disease = json_file['annotations']['disease']
    risk = json_file['annotations']['risk']
    
    label = f'{crop}_{disease}_{risk}'
    label_list.append(train_label_encoder[label])

100%|████████████████████████████████████████████████████████████████████████████| 5767/5767 [00:01<00:00, 3144.68it/s]


In [16]:
# train_transforms = A.Compose([
#                 A.Resize(288, 288),
#                 A.OneOf([
#                     A.Rotate(),
#                     A.HorizontalFlip(),
#                     A.VerticalFlip()
#                 ], p=1)
#             ])

# val_transforms = A.Compose([
#     A.Resize(288, 288)
# ])

# label_encoder = train_label_encoder

# data_list = glob('./data/train/*')
# print("total : ", len(data_list))

# train, val = train_test_split(data_list, test_size=0.2, shuffle=True, stratify=label_list)

# print("train : ", len(train))
# print("val : ", len(val))

# train_dataset = CustomDataset(
#             train, 
#             csv_feature_dict,
#             label_encoder,
#             transforms=train_transforms,
#         )
# val_dataset = CustomDataset(
#             val, 
#             csv_feature_dict,
#             label_encoder,
#             transforms=val_transforms,
#         )

# train_loader = DataLoader(
#     train_dataset, 
#     batch_size=batch_size, 
#     shuffle=True
# )

# val_loader = DataLoader(
#     val_dataset, 
#     batch_size=batch_size, 
#     shuffle=False
# )

In [13]:
train_transforms = A.Compose([
                A.Resize(288, 288),
                A.OneOf([
                    A.Rotate(),
                    A.HorizontalFlip(),
                    A.VerticalFlip()
                ], p=1)
            ])

val_transforms = A.Compose([
    A.Resize(288, 288)
])
label_encoder = train_label_encoder

In [14]:
data_list = glob('./data/train/*')
kfold = StratifiedKFold(n_splits=4, random_state=13, shuffle=True)

# K fold Training

In [19]:
for k, (fold_train, fold_val) in enumerate(kfold.split(data_list, label_list), 1) :
    model = CNN2RNNModel(model_path, n_classes)
    model = model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=5e-2)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20, eta_min=0.0001) 
    criterion = nn.CrossEntropyLoss()
    
    train_data_list = []
    val_data_list = []    
    for k_train in fold_train :
        train_data_list.append(data_list[k_train])
    
    for k_val in fold_val :
        val_data_list.append(data_list[k_val])
        
        
    print(f"\n\n\n===== k_fold : {k} / {fold_n} =====")
    train_dataset = CustomDataset(
                train_data_list, 
                csv_feature_dict,
                label_encoder,
                transforms=train_transforms,
            )
    val_dataset = CustomDataset(
                val_data_list, 
                csv_feature_dict,
                label_encoder,
                transforms=val_transforms,
            )

    train_loader = DataLoader(
        train_dataset, 
        batch_size=batch_size, 
        shuffle=True
    )

    val_loader = DataLoader(
        val_dataset, 
        batch_size=batch_size, 
        shuffle=False
    )

    loss_plot, val_loss_plot = [], []
    metric_plot, val_metric_plot = [], []

    early_stopping = 0
    for epoch in range(epochs):
        total_loss, total_val_loss = 0, 0
        total_acc, total_val_acc = 0, 0

        tqdm_dataset = tqdm(enumerate(train_loader))
        training = True
        for batch, batch_item in tqdm_dataset:
            batch_loss, batch_acc = train_step(batch_item, training)
            total_loss += batch_loss
            total_acc += batch_acc

            tqdm_dataset.set_postfix({
                'Epoch': epoch + 1,
                'Loss': '{:06f}'.format(batch_loss.item()),
                'Mean Loss' : '{:06f}'.format(total_loss/(batch+1)),
                'Mean F-1' : '{:06f}'.format(total_acc/(batch+1))
            })
        loss_plot.append(total_loss/(batch+1))
        metric_plot.append(total_acc/(batch+1))

        tqdm_dataset = tqdm(enumerate(val_loader))
        training = False
        for batch, batch_item in tqdm_dataset:
            batch_loss, batch_acc = train_step(batch_item, training)
            total_val_loss += batch_loss
            total_val_acc += batch_acc

            tqdm_dataset.set_postfix({
                'Epoch': epoch + 1,
                'Val Loss': '{:06f}'.format(batch_loss.item()),
                'Mean Val Loss' : '{:06f}'.format(total_val_loss/(batch+1)),
                'Mean Val F-1' : '{:06f}'.format(total_val_acc/(batch+1))
            })
        val_loss_plot.append(total_val_loss/(batch+1))
        val_metric_plot.append(total_val_acc/(batch+1))
        
        scheduler.step()
        
        if np.max(val_metric_plot) == val_metric_plot[-1]:
            torch.save(model.state_dict(), f'{k}_{save_path}')
            early_stopping = 0

        elif np.max(val_metric_plot) > val_metric_plot[-1]: 
            early_stopping += 1
            print(f"Early Stopping Step : [{early_stopping} / {early_stopping_cnt}]")

        if early_stopping == early_stopping_cnt :
            print("== Early Stop ==")
            break




===== k_fold : 1 / 4 =====


271it [02:40,  1.69it/s, Epoch=1, Loss=0.696115, Mean Loss=1.431567, Mean F-1=0.474266]
91it [00:42,  2.14it/s, Epoch=1, Val Loss=1.955474, Mean Val Loss=0.416796, Mean Val F-1=0.719961]
271it [02:34,  1.75it/s, Epoch=2, Loss=1.739481, Mean Loss=1.166044, Mean F-1=0.578668]
91it [00:42,  2.14it/s, Epoch=2, Val Loss=0.519295, Mean Val Loss=0.483337, Mean Val F-1=0.736528]
271it [02:35,  1.74it/s, Epoch=3, Loss=0.724580, Mean Loss=1.048112, Mean F-1=0.592015]
91it [00:45,  2.02it/s, Epoch=3, Val Loss=0.570880, Mean Val Loss=0.316277, Mean Val F-1=0.776346]
271it [02:40,  1.69it/s, Epoch=4, Loss=1.174053, Mean Loss=1.017561, Mean F-1=0.596064]
91it [00:44,  2.05it/s, Epoch=4, Val Loss=0.043030, Mean Val Loss=0.270884, Mean Val F-1=0.775114]


Early Stopping Step : [1 / 4]


271it [02:41,  1.68it/s, Epoch=5, Loss=1.154722, Mean Loss=0.945627, Mean F-1=0.622562]
91it [00:44,  2.06it/s, Epoch=5, Val Loss=1.275321, Mean Val Loss=0.219081, Mean Val F-1=0.830978]
271it [02:41,  1.68it/s, Epoch=6, Loss=0.712279, Mean Loss=0.909005, Mean F-1=0.646492]
91it [00:44,  2.04it/s, Epoch=6, Val Loss=0.721532, Mean Val Loss=0.180704, Mean Val F-1=0.849334]
271it [02:42,  1.66it/s, Epoch=7, Loss=0.962823, Mean Loss=0.855021, Mean F-1=0.689815]
91it [00:45,  2.01it/s, Epoch=7, Val Loss=0.055825, Mean Val Loss=0.150960, Mean Val F-1=0.859999]
271it [02:45,  1.64it/s, Epoch=8, Loss=1.318704, Mean Loss=0.845946, Mean F-1=0.651970]
91it [00:46,  1.97it/s, Epoch=8, Val Loss=0.341998, Mean Val Loss=0.157554, Mean Val F-1=0.870604]
271it [02:48,  1.61it/s, Epoch=9, Loss=0.339760, Mean Loss=0.780437, Mean F-1=0.695388]
91it [00:48,  1.89it/s, Epoch=9, Val Loss=1.426675, Mean Val Loss=0.154697, Mean Val F-1=0.883031]
271it [02:50,  1.59it/s, Epoch=10, Loss=0.869062, Mean Loss=0.797

Early Stopping Step : [1 / 4]


...

Early Stopping Step : [2 / 4]


271it [03:32,  1.27it/s, Epoch=14, Loss=0.374410, Mean Loss=0.654846, Mean F-1=0.741451]
91it [01:02,  1.45it/s, Epoch=14, Val Loss=0.003870, Mean Val Loss=0.208717, Mean Val F-1=0.888049]


Early Stopping Step : [3 / 4]


271it [03:39,  1.23it/s, Epoch=15, Loss=0.651731, Mean Loss=0.636249, Mean F-1=0.724978]
91it [01:02,  1.45it/s, Epoch=15, Val Loss=0.011037, Mean Val Loss=0.217308, Mean Val F-1=0.899436]

Early Stopping Step : [4 / 4]
== Early Stop ==


# training

In [ ]:
# loss_plot, val_loss_plot = [], []
# metric_plot, val_metric_plot = [], []

# early_stopping = 0
# for epoch in range(epochs):
#     total_loss, total_val_loss = 0, 0
#     total_acc, total_val_acc = 0, 0
    
#     tqdm_dataset = tqdm(enumerate(train_loader))
#     training = True
#     for batch, batch_item in tqdm_dataset:
#         batch_loss, batch_acc = train_step(batch_item, training)
#         total_loss += batch_loss
#         total_acc += batch_acc
        
#         tqdm_dataset.set_postfix({
#             'Epoch': epoch + 1,
#             'Loss': '{:06f}'.format(batch_loss.item()),
#             'Mean Loss' : '{:06f}'.format(total_loss/(batch+1)),
#             'Mean F-1' : '{:06f}'.format(total_acc/(batch+1))
#         })
#     loss_plot.append(total_loss/(batch+1))
#     metric_plot.append(total_acc/(batch+1))
    
#     tqdm_dataset = tqdm(enumerate(val_loader))
#     training = False
#     for batch, batch_item in tqdm_dataset:
#         batch_loss, batch_acc = train_step(batch_item, training)
#         total_val_loss += batch_loss
#         total_val_acc += batch_acc
        
#         tqdm_dataset.set_postfix({
#             'Epoch': epoch + 1,
#             'Val Loss': '{:06f}'.format(batch_loss.item()),
#             'Mean Val Loss' : '{:06f}'.format(total_val_loss/(batch+1)),
#             'Mean Val F-1' : '{:06f}'.format(total_val_acc/(batch+1))
#         })
#     val_loss_plot.append(total_val_loss/(batch+1))
#     val_metric_plot.append(total_val_acc/(batch+1))
    
#     if np.max(val_metric_plot) == val_metric_plot[-1]:
#         torch.save(model.state_dict(), save_path)
        
#     elif np.max(val_metric_plot) > val_metric_plot[-1]: 
#         early_stopping += 1
#         print(f"Early Stopping Step : [{early_stopping} / 4]")

#     if early_stopping == early_stopping_cnt :
#         print("== Early Stop ==")
#         break

In [16]:
class _pretrained_EffiV2S(nn.Module):
    def __init__(self):
        super(_pretrained_EffiV2S, self).__init__()
        self.model = timm.create_model('efficientnetv2_rw_s', num_classes=38, pretrained=False)
    
    def forward(self, inputs):
        output = self.model(inputs)
        return output
    
class _EffiV2S(nn.Module) :
    def __init__(self, model_path, test=False) :
        super(_EffiV2S, self).__init__()
        self.pre_model = _pretrained_EffiV2S()
        if test == False :
            self.pre_model.load_state_dict(torch.load(model_path, map_location=device))
        
        self.fc = nn.Linear(38, 25)
        
    def forward(self, x) :
        x = self.pre_model(x)
        x = self.fc(x)
        return x   


# Test Dataset 정의

In [22]:
val_transforms = A.Compose([
    A.Resize(288,288)
])

test = sorted(glob('data/test/*'))
test_dataset = CustomDataset(
            test, 
            csv_feature_dict,
            label_encoder,
            transforms=val_transforms,
            mode = 'test'
        )
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [17]:
import torch.nn.functional as F

def softvoting(models, item, n_classes=25) :
    img = item['img'].to(device)
    seq_len = item['seq_len'].to(device)
    csv_feature = item['csv_feature'].to(device)
    
    predicts = torch.zeros(img.size(0), n_classes)
    with torch.no_grad() :
        for idx, model in enumerate(models) :
            if idx < 4 :
                output = model(img, csv_feature, seq_len)
            elif idx > 3 :
                output = model(img)
            output = F.softmax(output.cpu(), dim=1)
            predicts += output

    # 둘다 값은 똑같이 나옴.
    # pred_avg = predicts / len(models)
    # answer = pred_avg.argmax(dim=-1)
    # _, answer2 = torch.max(pred_avg, 1)

    return predicts.detach().cpu() / len(models)



In [18]:
def predict(dataset, models) :
    tqdm_dataset = tqdm(enumerate(dataset))
    results = []
    for batch, batch_item in tqdm_dataset :
        item = batch_item
#         print(img.shape)
        predictions = softvoting(models, item)
        batch_result = [int(torch.argmax(prediction)) for prediction in predictions]
#         print(batch_result)
#         for prediction in predictions :
            
#         results.append(int(torch.argmax(predictions[0])))
#             output = model(img)
#         output = torch.tensor(torch.argmax(output, dim=1), dtype=torch.int32).cpu().numpy()
        results.extend(batch_result)
    return results

kfold_models_path = glob('./model/k_public_vill_50k_pretrain_Effi_1000_CNN2RNN/*.pt') + glob('./model/k_fold_50k_pretrained_effiv2S/*.pt')
models = []
for idx, kfold_model_path in enumerate(kfold_models_path) :
    if idx < 4 :
        model = CNN2RNNModel(model_path, n_classes)    
    elif idx > 3 :
        model = _EffiV2S('',test=True)

    model.load_state_dict(torch.load(kfold_model_path, map_location=device))
    model.to(device).eval()
    models.append(model)

    
preds = predict(test_dataloader, models)




3245it [49:09,  1.10it/s]


In [19]:
preds_cp = preds

In [20]:
preds_cp = np.array([train_label_decoder[int(val)] for val in preds_cp])
submission_csv = pd.read_csv('./data/sample_submission.csv')
submission_csv['label'] = preds_cp
submission_csv.to_csv('./data/k_public_vill_50k_pretrain_KFOLD_Effi_Effi_1000_CNN2RNN.csv', index=False)


# hard Voting

In [25]:
def hardvoting(models, item) :
    img = item['img'].to(device)
    seq_len = item['seq_len'].to(device)
    csv_feature = item['csv_feature'].to(device)
    
    predicts = torch.zeros(img.size(0), n_classes)
    results = {}
    tmp_vals = [0] * 16
    tmp_inds = [0] * 16
    with torch.no_grad() :
        for idx, model in enumerate(models) :
            if idx < 4 :
                outputs = model(img, csv_feature, seq_len)
            elif idx > 3 :
                outputs = model(img)
                
#             outputs = model(img, csv_feature, seq_len)
            outputs = F.softmax(outputs.cpu(), dim=1)

            vals, indices = torch.max(outputs, 1)
            
            for i, (val, idx) in enumerate(zip(vals, indices)) :
                if tmp_vals[i] < val :
                    tmp_vals[i] = val.item()
                    tmp_inds[i] = idx.item()
                    
                
#             print(tmp_vals)
#             print(tmp_inds)


            predicts += outputs

    # 둘다 값은 똑같이 나옴.
    # pred_avg = predicts / len(models)
    # answer = pred_avg.argmax(dim=-1)
    # _, answer2 = torch.max(pred_avg, 1)

    return predicts.detach().cpu() / len(models)

In [26]:
def predict(dataset, models) :
    tqdm_dataset = tqdm(enumerate(dataset))
    results = []
    for batch, batch_item in tqdm_dataset :
        item = batch_item
#         print(img.shape)
        predictions = hardvoting(models, item)
        batch_result = [int(torch.argmax(prediction)) for prediction in predictions]
#         print(batch_result)
#         for prediction in predictions :
            
#         results.append(int(torch.argmax(predictions[0])))
#             output = model(img)
#         output = torch.tensor(torch.argmax(output, dim=1), dtype=torch.int32).cpu().numpy()
        results.extend(batch_result)
    return results

# kfold_models_path = glob('./model/k_public_vill_50k_pretrain_Effi_1000_CNN2RNN/*.pt')
kfold_models_path = glob('./model/k_public_vill_50k_pretrain_Effi_1000_CNN2RNN/*.pt') + glob('./model/k_fold_50k_pretrained_effiv2S/*.pt')
models = []
for idx, kfold_model_path in enumerate(kfold_models_path) :
    if idx < 4 :
        model = CNN2RNNModel(model_path, n_classes)    
    elif idx > 3 :
        model = _EffiV2S('',test=True)
#     model = CNN2RNNModel(model_path, n_classes) 

    model.load_state_dict(torch.load(kfold_model_path, map_location=device))
    model.to(device).eval()
    models.append(model)

    
preds = predict(test_dataloader, models)




3245it [50:23,  1.07it/s]


In [27]:
preds_cp = preds

In [28]:
preds_cp = np.array([train_label_decoder[int(val)] for val in preds_cp])
submission_csv = pd.read_csv('./data/sample_submission.csv')
submission_csv['label'] = preds_cp
submission_csv.to_csv('./data/k_public_vill_50k_pretrain_KFOLD_Effi_Effi_HardVoting_1000_CNN2RNN.csv', index=False)
